# Blur

In [6]:
import cv2
import os
import numpy as np

def apply_motion_blur(image, kernel_size=15):
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[int((kernel_size - 1)/2), :] = np.ones(kernel_size)
    kernel = kernel / kernel_size

    blurred = cv2.filter2D(image, -1, kernel)
    return blurred

def process_images(input_folder, output_folder, kernel_size=15):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):

            img_path = os.path.join(input_folder, filename)
            image = cv2.imread(img_path)

            blurred_image = apply_motion_blur(image, kernel_size)

            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, blurred_image)

            print(f"Processed and saved: {output_path}")

input_folder = 'datasetfull/train/blur'
output_folder = 'datasetfull/train/input'

process_images(input_folder, output_folder, kernel_size=15)

Processed and saved: datasetfull/train/input\finetune_img_0000_000.png
Processed and saved: datasetfull/train/input\finetune_img_0000_001.png
Processed and saved: datasetfull/train/input\finetune_img_0000_002.png
Processed and saved: datasetfull/train/input\finetune_img_0000_003.png
Processed and saved: datasetfull/train/input\finetune_img_0000_004.png
Processed and saved: datasetfull/train/input\finetune_img_0000_005.png
Processed and saved: datasetfull/train/input\finetune_img_0000_006.png
Processed and saved: datasetfull/train/input\finetune_img_0000_007.png
Processed and saved: datasetfull/train/input\finetune_img_0000_008.png
Processed and saved: datasetfull/train/input\finetune_img_0000_009.png
Processed and saved: datasetfull/train/input\finetune_img_0000_010.png
Processed and saved: datasetfull/train/input\finetune_img_0000_011.png
Processed and saved: datasetfull/train/input\finetune_img_0000_012.png
Processed and saved: datasetfull/train/input\finetune_img_0000_013.png
Proces

# Load Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image
import os
from runpy import run_path
from skimage import img_as_ubyte
from collections import OrderedDict
from natsort import natsorted
from glob import glob
import cv2
import argparse

# parser = argparse.ArgumentParser(description='Demo MPRNet')
# parser.add_argument('--input_dir', default='Kvasir-SEG-Blured/images/val', type=str, help='Input images')
# parser.add_argument('--result_dir', default='Kvasir-SEG-DeBlured/images/val', type=str, help='Directory for results')
# parser.add_argument('--task', required=True, type=str, help='Task to run', choices=['Deblurring', 'Denoising', 'Deraining'])

# args = parser.parse_args()

def save_img(filepath, img):
    cv2.imwrite(filepath,cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

def load_checkpoint(model, weights):
    checkpoint = torch.load(weights)
    try:
        model.load_state_dict(checkpoint["state_dict"])
    except:
        state_dict = checkpoint["state_dict"]
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:] 
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)


In [ ]:
task    = 'Deblurring'
inp_dir = 'Kvasir-SEG-Blured/images/val'
out_dir = 'Kvasir-SEG-DeBlured/images/val'



os.makedirs(out_dir, exist_ok=True)

files = natsorted(glob(os.path.join(inp_dir, '*.jpg'))
                + glob(os.path.join(inp_dir, '*.JPG'))
                + glob(os.path.join(inp_dir, '*.png'))
                + glob(os.path.join(inp_dir, '*.PNG')))

if len(files) == 0:
    raise Exception(f"No files found at {inp_dir}")

load_file = run_path(os.path.join(task, "MPRNet.py"))
model = load_file['MPRNet']()
model.cuda()

weights = os.path.join(task, "pretrained_models", "model_"+task.lower()+".pth")
load_checkpoint(model, weights)
model.eval()

img_multiple_of = 8

for file_ in files:
    img = Image.open(file_).convert('RGB')
    input_ = TF.to_tensor(img).unsqueeze(0).cuda()

    h,w = input_.shape[2], input_.shape[3]
    H,W = ((h+img_multiple_of)//img_multiple_of)*img_multiple_of, ((w+img_multiple_of)//img_multiple_of)*img_multiple_of
    padh = H-h if h%img_multiple_of!=0 else 0
    padw = W-w if w%img_multiple_of!=0 else 0
    input_ = F.pad(input_, (0,padw,0,padh), 'reflect')

    with torch.no_grad():
        restored = model(input_)
    restored = restored[0]
    restored = torch.clamp(restored, 0, 1)

    restored = restored[:,:,:h,:w]

    restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
    restored = img_as_ubyte(restored[0])

    f = os.path.splitext(os.path.split(file_)[-1])[0]
    save_img((os.path.join(out_dir, f+'.png')), restored)

print(f"Files saved at {out_dir}")


# Low Res

In [6]:
import cv2
import os

input_folder = 'noisy/images/train'  
output_folder = 'noisy_denoised/images/train' 

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
        file_path = os.path.join(input_folder, filename)
        
        img = cv2.imread(file_path)
        
        resized_img = cv2.resize(img, (50, 50))
        
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, resized_img)
        
        print(f"Resized and saved: {filename}")

print("All images have been resized and saved to the new folder.")

Resized and saved: train_img_0000_000.png
Resized and saved: train_img_0000_001.png
Resized and saved: train_img_0000_002.png
Resized and saved: train_img_0000_003.png
Resized and saved: train_img_0000_004.png
Resized and saved: train_img_0000_005.png
Resized and saved: train_img_0000_006.png
Resized and saved: train_img_0000_007.png
Resized and saved: train_img_0000_008.png
Resized and saved: train_img_0000_009.png
Resized and saved: train_img_0000_010.png
Resized and saved: train_img_0000_011.png
Resized and saved: train_img_0000_012.png
Resized and saved: train_img_0000_013.png
Resized and saved: train_img_0000_014.png
Resized and saved: train_img_0000_015.png
Resized and saved: train_img_0000_016.png
Resized and saved: train_img_0000_017.png
Resized and saved: train_img_0000_018.png
Resized and saved: train_img_0000_019.png
Resized and saved: train_img_0000_020.png
Resized and saved: train_img_0000_021.png
Resized and saved: train_img_0000_022.png
Resized and saved: train_img_0000_

# Upscaling.


In [1]:
import torch
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
from PIL import Image
import glob
import os
import numpy as np

model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64,
                num_block=23, num_grow_ch=32, scale=4)
upscaler = RealESRGANer(scale=4, model_path='RealESRGAN_x4plus.pth', model=model, tile=0, tile_pad=10, pre_pad=0, half=True)

# input_folder = 'Kvasir-SEG/images/train'
input_folder = 'images/train'
output_folder = 'imagesx4/train'
os.makedirs(output_folder, exist_ok=True)

image_paths = glob.glob(os.path.join(input_folder, '*'))

for image_path in image_paths:
    img = Image.open(image_path).convert('RGB')
    img = np.array(img)

    with torch.no_grad():
        upscaled_img, _ = upscaler.enhance(img, outscale=4)

    upscaled_img = Image.fromarray(upscaled_img)
    base_name = os.path.basename(image_path)
    upscaled_img.save(os.path.join(output_folder, base_name))


c:\Users\Ben\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [2]:
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64,
                num_block=23, num_grow_ch=32, scale=4)
upscaler = RealESRGANer(scale=4, model_path='RealESRGAN_x4plus.pth', model=model, tile=0, tile_pad=10, pre_pad=0, half=True)

# input_folder = 'Kvasir-SEG/images/train'
input_folder = 'images/val'
output_folder = 'imagesx4/val'
os.makedirs(output_folder, exist_ok=True)

image_paths = glob.glob(os.path.join(input_folder, '*'))

for image_path in image_paths:
    img = Image.open(image_path).convert('RGB')
    img = np.array(img)

    with torch.no_grad():
        upscaled_img, _ = upscaler.enhance(img, outscale=4)

    upscaled_img = Image.fromarray(upscaled_img)
    base_name = os.path.basename(image_path)
    upscaled_img.save(os.path.join(output_folder, base_name))


In [9]:
import shutil


def combine_folders_with_subfolders(top_level_folders, output_folder='all'):
    base_dir = os.getcwd()

    output_images_train = os.path.join(base_dir, output_folder, 'images', 'train')
    output_images_val = os.path.join(base_dir, output_folder, 'images', 'val')
    output_labels_train = os.path.join(base_dir, output_folder, 'labels', 'train')
    output_labels_val = os.path.join(base_dir, output_folder, 'labels', 'val')

    os.makedirs(output_images_train, exist_ok=True)
    os.makedirs(output_images_val, exist_ok=True)
    os.makedirs(output_labels_train, exist_ok=True)
    os.makedirs(output_labels_val, exist_ok=True)

    for folder in top_level_folders:
        images_train_folder = os.path.join(base_dir, folder, 'images', 'train')
        images_val_folder = os.path.join(base_dir, folder, 'images', 'val')
        labels_train_folder = os.path.join(base_dir, folder, 'labels', 'train')
        labels_val_folder = os.path.join(base_dir, folder, 'labels', 'val')

        if os.path.exists(images_train_folder):
            for file_name in os.listdir(images_train_folder):
                src_file = os.path.join(images_train_folder, file_name)
                new_file_name = f"{folder}_{file_name}"
                dest_file = os.path.join(output_images_train, new_file_name)
                shutil.copy(src_file, dest_file)
                print(f"Copied {src_file} to {dest_file}")

        if os.path.exists(images_val_folder):
            for file_name in os.listdir(images_val_folder):
                src_file = os.path.join(images_val_folder, file_name)
                new_file_name = f"{folder}_{file_name}"
                dest_file = os.path.join(output_images_val, new_file_name)
                shutil.copy(src_file, dest_file)
                print(f"Copied {src_file} to {dest_file}")

        if os.path.exists(labels_train_folder):
            for file_name in os.listdir(labels_train_folder):
                src_file = os.path.join(labels_train_folder, file_name)
                new_file_name = f"{folder}_{file_name}"
                dest_file = os.path.join(output_labels_train, new_file_name)
                shutil.copy(src_file, dest_file)
                print(f"Copied {src_file} to {dest_file}")

        if os.path.exists(labels_val_folder):
            for file_name in os.listdir(labels_val_folder):
                src_file = os.path.join(labels_val_folder, file_name)
                new_file_name = f"{folder}_{file_name}"
                dest_file = os.path.join(output_labels_val, new_file_name)
                shutil.copy(src_file, dest_file)
                print(f"Copied {src_file} to {dest_file}")

In [10]:
top_level_folders = ["noisy", "normal"]  

combine_folders_with_subfolders(top_level_folders)


Copied c:\Users\Ben\Documents\WorkComp\Uoa Engineering Master\Year 4\Sem2\760\Project\compsci-760-project\noisy\images\train\train_img_0000_000.png to c:\Users\Ben\Documents\WorkComp\Uoa Engineering Master\Year 4\Sem2\760\Project\compsci-760-project\all\images\train\noisy_train_img_0000_000.png
Copied c:\Users\Ben\Documents\WorkComp\Uoa Engineering Master\Year 4\Sem2\760\Project\compsci-760-project\noisy\images\train\train_img_0000_001.png to c:\Users\Ben\Documents\WorkComp\Uoa Engineering Master\Year 4\Sem2\760\Project\compsci-760-project\all\images\train\noisy_train_img_0000_001.png
Copied c:\Users\Ben\Documents\WorkComp\Uoa Engineering Master\Year 4\Sem2\760\Project\compsci-760-project\noisy\images\train\train_img_0000_002.png to c:\Users\Ben\Documents\WorkComp\Uoa Engineering Master\Year 4\Sem2\760\Project\compsci-760-project\all\images\train\noisy_train_img_0000_002.png
Copied c:\Users\Ben\Documents\WorkComp\Uoa Engineering Master\Year 4\Sem2\760\Project\compsci-760-project\noisy\